# Capstone Final Project - Battle of the Neighborhoods

### Applied Data Science Capstone IBM/Coursera Course

## Table of Contents

 * [Introduction](#introduction)
 * [Data](#data)
 * [Methods](#methods)
 * [Analysis](#analysis)
 * [Results and Discussion](#results)
 * [Conclusion](#conclusion)

# Introduction <a name="introduction"></a>
The idea of this study is to clearly settle, once and for all, which city is best: San Francisco or New York City. I will frame this study by exploring which city gives "more bang for your buck". I will answer this question by comparing median rent prices of neighborhoods in each city (the buck), and venues that reside in these neighborhoods (the bang). This study will give clarity to individuals in both cities that are considering moving to the other. Additionally, it will inform current residents of San Francisco and New York City, of other neighborhoods within their own city they could potentially move to whether they want to pay less in rent or live near certain venues. Examples of this could be a family wanting to move to an area with more parks and playgrounds, a coffee enthusiast wanting to live near the most coffee shops as possible, or your average person looking to pay less rent in a neighborhood comparable to their current one.

# Data <a name="data"></a>


# Methods <a name="methods"></a>
## Cleaning Datasets

#### Import packages to clean neighborhood and transportation data.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Defining file paths so each csv can be read into a Pandas dataframe

In [2]:
#sf rent/location
file_path = 'https://raw.githubusercontent.com/d-alvear/Coursera_Capstone/master/data/sf_rent.csv'

#nyc rent/location
file_path2 = 'https://raw.githubusercontent.com/d-alvear/Coursera_Capstone/master/data/medianrent_1B_NYC.csv'

#### NYC and SF datasets only need columns to be renamed

In [3]:
df_sfRent = pd.read_csv(file_path)
df_sfRent.head()
#dataframe looks good, just rename some of the columns
df_sfRent.rename(columns={'MedianRent':'Rent', 'Lat':'Latitude', 'Long':'Longitude'}, inplace = True)

In [4]:
df_nycRent = pd.read_csv(file_path2)
df_nycRent.head()
#dataframe looks good, just rename some of the columns
df_nycRent.rename(columns={'MedianRent':'Rent', 'Lat':'Latitude', 'Long':'Longitude'}, inplace = True)

In [5]:
#df_nycRent.to_csv('nyc_rent.csv')
#df_sfRent.to_csv('sf_rent.csv')

#### Now I will create a map of San Francisco with neighborhood markers

In [6]:
import folium
from folium import plugins

In [7]:
latitude = 37.7749 
longitude = -122.4194
# creating map of SF using latitude and longitude values
map_sf = folium.Map(location=[latitude, longitude], zoom_start=12.25)

# add markers to map
for lat, lng, neighborhood, rent in zip(df_sfRent['Latitude'], df_sfRent['Longitude'], df_sfRent['Neighborhood'], df_sfRent['Rent']):
    label = 'Neighborhood: {}, Median Rent: ${}'.format(neighborhood, rent)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sf)

map_sf

# Analysis <a name="analysis"></a>

In [8]:
CLIENT_ID = 'IW1DNBRJOUHECGXR0ST002ZPDJYOBCIRTROZZ5YJZFHCIP3Q' # your Foursquare ID
CLIENT_SECRET = 'TWMIYINYFXOLI0IGNPXXLCIRHFWLUNJ4VP2TJZXSJGUP2ZBN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IW1DNBRJOUHECGXR0ST002ZPDJYOBCIRTROZZ5YJZFHCIP3Q
CLIENT_SECRET:TWMIYINYFXOLI0IGNPXXLCIRHFWLUNJ4VP2TJZXSJGUP2ZBN


In [9]:
sf_hoods = df_sfRent[['Neighborhood', 'Latitude', 'Longitude']]
sf_hoods.head()

,Neighborhood,Latitude,Longitude
0,Outer Richmond,37.7777,-122.4953
1,Outer Sunset,37.7467,-122.4863
2,Lakeshore,37.7233,-122.4887
3,Inner Richmond,37.7810,-122.4654
4,Inner Sunset,37.7607,-122.4680


In [10]:
neighborhood_latitude = sf_hoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = sf_hoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = sf_hoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Outer Richmond are 37.7777, -122.4953.


In [11]:
# type your answer here
LIMIT = 50
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=IW1DNBRJOUHECGXR0ST002ZPDJYOBCIRTROZZ5YJZFHCIP3Q&client_secret=TWMIYINYFXOLI0IGNPXXLCIRHFWLUNJ4VP2TJZXSJGUP2ZBN&v=20180605&ll=37.7777,-122.4953&radius=500&limit=50'

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d22787d86bc49002c2198ff'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Outer Richmond',
  'headerFullLocation': 'Outer Richmond, San Francisco',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 53,
  'suggestedBounds': {'ne': {'lat': 37.782200004500005,
    'lng': -122.48961725783376},
   'sw': {'lat': 37.7731999955, 'lng': -122.50098274216624}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5941f709898bdc060880ee9c',
       'name': 'Jiangnan Cuisine',
       'location': {'address': '3420 Balboa St',
        'crossStreet': '35th Ave',
        'lat': 37.77588961511461,
        'lng': 

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Jiangnan Cuisine,Chinese Restaurant,37.775890,-122.495616
1,Cassava,New American Restaurant,37.775722,-122.496702
2,Mr. Bánh Mì,Sandwich Place,37.775700,-122.497491
3,Simple Pleasures Cafe,Café,37.775908,-122.495810
4,Pacific Cafe,Seafood Restaurant,37.779782,-122.494428


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

50 venues were returned by Foursquare.


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
sf_venues = getNearbyVenues(names=sf_hoods['Neighborhood'],
                                   latitudes=sf_hoods['Latitude'],
                                   longitudes=sf_hoods['Longitude']
                                  )


Outer Richmond
Outer Sunset
Lakeshore
Inner Richmond
Inner Sunset
West of Twin Peaks
Presidio Heights/Laurel Heights
Lone Mountain
NOPA
Haight Ashbury
Ashbury Heights
The Castro
Noe Valley
Glen Park
Outer Mission/Excelsior
Marina
Cow Hollow
Pacific Heights
Lower Pacific Heights
Western Addition
Lower Haight
Mission Dolores
Hayes Valley
North Beach
Russian Hill
Nob Hill
Downtown
Tenderloin
Civic Center
SOMA
The Mission
Bernal Heights
Telegraph Hill
Financial District
South Beach
Protrero Hill
Mission Bay/Dogpatch
Bayview


In [18]:
print(sf_venues.shape)
sf_venues.head()

(1584, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Outer Richmond,37.7777,-122.4953,Jiangnan Cuisine,37.775890,-122.495616,Chinese Restaurant
1,Outer Richmond,37.7777,-122.4953,Cassava,37.775722,-122.496702,New American Restaurant
2,Outer Richmond,37.7777,-122.4953,Mr. Bánh Mì,37.775700,-122.497491,Sandwich Place
3,Outer Richmond,37.7777,-122.4953,Simple Pleasures Cafe,37.775908,-122.495810,Café
4,Outer Richmond,37.7777,-122.4953,Pacific Cafe,37.779782,-122.494428,Seafood Restaurant


In [19]:
# one hot encoding
sf_onehot = pd.get_dummies(sf_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sf_onehot['Neighborhood'] = sf_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sf_onehot.columns[0]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns]

sf_onehot.head()

,Acai House,Acai House,Accessories Store,Afghan Restaurant,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Board Shop,Bookstore,Border Crossing,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Field,Filipino Restaurant,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hawaiian Restaurant,Health & Beauty Service,Hill,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Moving Target,Music School,Music Store,Music Venue,Nabe Restaurant,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Toy / Game Store,Trade School,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Outer Richmond,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [20]:
sf_grouped = sf_onehot.groupby('Neighborhood').mean().reset_index()
sf_grouped

,Neighborhood,Acai House,Acai House,Accessories Store,Afghan Restaurant,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Board Shop,Bookstore,Border Crossing,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Field,Filipino Restaurant,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hawaiian Restaurant,Health & Beauty Service,Hill,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Moving Target,Music School,Music Store,Music Venue,Nabe Restaurant,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Toy / Game Store,Trade School,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,Ashbury Heights,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.025641,0.000000,0.025641,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00000,0.00,0.076923,0.000000,0.000000,0.00,0.025641,0.00,0.000000,0.000000,0.076923,0.000000,0.000000,0.025641,0.00,0.00,0.00,0.000000,0.025641,0.000000,0.00,0.00,0.00,0.00,0.000000,0.051282,0.000000,0.00000

In [21]:
num_top_venues = 5

for hood in sf_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = sf_grouped[sf_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ashbury Heights----
                venue  freq
0      Breakfast Spot  0.08
1         Bus Station  0.08
2  Italian Restaurant  0.05
3   Convenience Store  0.05
4         Coffee Shop  0.05


----Bayview----
                     venue  freq
0                  Brewery  0.33
1              Bus Station  0.33
2                      Gym  0.33
3  New American Restaurant  0.00
4                Newsstand  0.00


----Bernal Heights----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.05
2        Gourmet Shop  0.05
3              Bakery  0.05
4                Park  0.05


----Civic Center----
                   venue  freq
0  Vietnamese Restaurant  0.12
1         Sandwich Place  0.06
2        Thai Restaurant  0.06
3                Theater  0.06
4               Beer Bar  0.06


----Cow Hollow----
                venue  freq
0   French Restaurant  0.08
1  Italian Restaurant  0.08
2     Thai Restaurant  0.06
3            Wine Bar  0.06
4       Deli / Bodega  0.04



In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sf_grouped['Neighborhood']

for ind in np.arange(sf_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sf_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Ashbury Heights,Breakfast Spot,Bus Station,Italian Restaurant,Trail,Convenience Store
1,Bayview,Brewery,Bus Station,Gym,Women's Store,Frozen Yogurt Shop
2,Bernal Heights,Coffee Shop,Italian Restaurant,Gourmet Shop,Bakery,Park
3,Civic Center,Vietnamese Restaurant,Thai Restaurant,Theater,Sandwich Place,Beer Bar
4,Cow Hollow,Italian Restaurant,French Restaurant,Thai Restaurant,Wine Bar,Gym / Fitness Center
5,Downtown,Hotel,Italian Restaurant,Bar,Grocery Store,Beer Bar
6,Financial District,Sandwich Place,Italian Restaurant,Deli / Bodega,Salad Place,Sushi Restaurant
7,Glen Park,Trail,Coffee Shop,Italian Restaurant,Café,Mexican Restaurant
8,Haight Ashbury,Thrift / Vintage Store,Shoe Store,Gift Shop,Bookstore,Boutique
9,Hayes Valley,Wine Bar,French Restaurant,Sushi Restaurant,Dessert Shop,Optical Shop


In [24]:
# set number of clusters
kclusters = 10

sf_grouped_clustering = sf_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sf_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 3, 9, 5, 3, 5, 3, 3, 5], dtype=int32)

In [25]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

sf_merged = sf_hoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sf_merged = sf_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

sf_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Outer Richmond,37.7777,-122.4953,3,Café,Chinese Restaurant,Bakery,Sandwich Place,Japanese Restaurant
1,Outer Sunset,37.7467,-122.4863,7,Dumpling Restaurant,Chinese Restaurant,Korean Restaurant,Shoe Store,Gastropub
2,Lakeshore,37.7233,-122.4887,1,Golf Course,Pizza Place,Park,American Restaurant,College Cafeteria
3,Inner Richmond,37.7810,-122.4654,0,Japanese Restaurant,Bakery,Thai Restaurant,Korean Restaurant,Vietnamese Restaurant
4,Inner Sunset,37.7607,-122.4680,0,Ice Cream Shop,Coffee Shop,Bakery,Sandwich Place,Chinese Restaurant


In [26]:
sf_merged

,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Outer Richmond,37.7777,-122.4953,3,Café,Chinese Restaurant,Bakery,Sandwich Place,Japanese Restaurant
1,Outer Sunset,37.7467,-122.4863,7,Dumpling Restaurant,Chinese Restaurant,Korean Restaurant,Shoe Store,Gastropub
2,Lakeshore,37.7233,-122.4887,1,Golf Course,Pizza Place,Park,American Restaurant,College Cafeteria
3,Inner Richmond,37.7810,-122.4654,0,Japanese Restaurant,Bakery,Thai Restaurant,Korean Restaurant,Vietnamese Restaurant
4,Inner Sunset,37.7607,-122.4680,0,Ice Cream Shop,Coffee Shop,Bakery,Sandwich Place,Chinese Restaurant
5,West of Twin Peaks,37.7406,-122.4589,4,Park,Bus Stop,Jewelry Store,Monument / Landmark,Trail
6,Presidio Heights/Laurel Heights,37.7866,-122.4519,3,American Restaurant,Italian Restaurant,Café,Cosmetics Shop,Pet Store
7,Lone Mountain,37.7770,-122.4538,6,Café,Deli / Bodega,Women's Store,Bank,College Gym
8,NOPA,37.7757,-122.4425,3,Bar,Sushi Restaurant,Liquor Store,Mexican Restaurant,Café
9,Haight Ashbury,37.7692,-122.4481,3,Thrift / Vintage Store,Shoe Store,Gift Shop,Bookstore,Boutique


In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_merged['Latitude'], sf_merged['Longitude'], sf_merged['Neighborhood'], sf_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Results and Discussion <a name="results"></a>

# Conclusion <a name="conclusion"></a>

### Cluster 0

In [35]:
sf_merged.loc[sf_merged['Cluster Label'] == 0, sf_merged.columns[[0] + list(range(3, sf_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Inner Richmond,0,Japanese Restaurant,Bakery,Thai Restaurant,Korean Restaurant,Vietnamese Restaurant
4,Inner Sunset,0,Ice Cream Shop,Coffee Shop,Bakery,Sandwich Place,Chinese Restaurant
18,Lower Pacific Heights,0,Bakery,Spa,Ice Cream Shop,Boutique,Sandwich Place
19,Western Addition,0,Creperie,Tea Room,New American Restaurant,Gift Shop,Spa


### Cluster 1

In [36]:
sf_merged.loc[sf_merged['Cluster Label'] == 1, sf_merged.columns[[0] + list(range(3, sf_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Lakeshore,1,Golf Course,Pizza Place,Park,American Restaurant,College Cafeteria


### Cluster 2

In [37]:
sf_merged.loc[sf_merged['Cluster Label'] == 2, sf_merged.columns[[0] + list(range(3, sf_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,Bayview,2,Brewery,Bus Station,Gym,Women's Store,Frozen Yogurt Shop


### Cluster 3

In [38]:
sf_merged.loc[sf_merged['Cluster Label'] == 3, sf_merged.columns[[0] + list(range(3, sf_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Outer Richmond,3,Café,Chinese Restaurant,Bakery,Sandwich Place,Japanese Restaurant
6,Presidio Heights/Laurel Heights,3,American Restaurant,Italian Restaurant,Café,Cosmetics Shop,Pet Store
8,NOPA,3,Bar,Sushi Restaurant,Liquor Store,Mexican Restaurant,Café
9,Haight Ashbury,3,Thrift / Vintage Store,Shoe Store,Gift Shop,Bookstore,Boutique
10,Ashbury Heights,3,Breakfast Spot,Bus Station,Italian Restaurant,Trail,Convenience Store
12,Noe Valley,3,Coffee Shop,Gift Shop,Bakery,Italian Restaurant,American Restaurant
13,Glen Park,3,Trail,Coffee Shop,Italian Restaurant,Café,Mexican Restaurant
17,Pacific Heights,3,Park,Women's Store,Sandwich Place,Coffee Shop,Cosmetics Shop
20,Lower Haight,3,Coffee Shop,Boutique,Park,Café,Dive Bar
21,Mission Dolores,3,Gift Shop,Park,Playground,Juice Bar,Bar


### Cluster 4

In [39]:
sf_merged.loc[sf_merged['Cluster Label'] == 4, sf_merged.columns[[0] + list(range(3, sf_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,West of Twin Peaks,4,Park,Bus Stop,Jewelry Store,Monument / Landmark,Trail


### Cluster 5

In [40]:
sf_merged.loc[sf_merged['Cluster Label'] == 5, sf_merged.columns[[0] + list(range(3, sf_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,The Castro,5,Gay Bar,Thai Restaurant,Coffee Shop,Wine Bar,Deli / Bodega
15,Marina,5,Gym / Fitness Center,French Restaurant,Sushi Restaurant,Electronics Store,Wine Bar
16,Cow Hollow,5,Italian Restaurant,French Restaurant,Thai Restaurant,Wine Bar,Gym / Fitness Center
22,Hayes Valley,5,Wine Bar,French Restaurant,Sushi Restaurant,Dessert Shop,Optical Shop
24,Russian Hill,5,Park,Sushi Restaurant,Italian Restaurant,Pizza Place,Spa
25,Nob Hill,5,Italian Restaurant,Gym / Fitness Center,Deli / Bodega,Hotel,Wine Bar
33,Financial District,5,Sandwich Place,Italian Restaurant,Deli / Bodega,Salad Place,Sushi Restaurant
36,Mission Bay/Dogpatch,5,Gym / Fitness Center,Café,Coffee Shop,Cocktail Bar,Wine Bar


### Cluster 6

In [41]:
sf_merged.loc[sf_merged['Cluster Label'] == 6, sf_merged.columns[[0] + list(range(3, sf_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,Lone Mountain,6,Café,Deli / Bodega,Women's Store,Bank,College Gym


### Cluster 7

In [42]:
sf_merged.loc[sf_merged['Cluster Label'] == 7, sf_merged.columns[[0] + list(range(3, sf_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Outer Sunset,7,Dumpling Restaurant,Chinese Restaurant,Korean Restaurant,Shoe Store,Gastropub


### Cluster 8

In [43]:
sf_merged.loc[sf_merged['Cluster Label'] == 8, sf_merged.columns[[0] + list(range(3, sf_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,Outer Mission/Excelsior,8,Pizza Place,Vietnamese Restaurant,Mexican Restaurant,Men's Store,Chinese Restaurant


### Cluster 9

In [44]:
sf_merged.loc[sf_merged['Cluster Label'] == 9, sf_merged.columns[[0] + list(range(3, sf_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
27,Tenderloin,9,Vietnamese Restaurant,Coffee Shop,Thai Restaurant,Speakeasy,Cocktail Bar
28,Civic Center,9,Vietnamese Restaurant,Thai Restaurant,Theater,Sandwich Place,Beer Bar
29,SOMA,9,Coffee Shop,Vietnamese Restaurant,Wine Bar,Bakery,Bar


### Cluster 10

In [45]:
sf_merged.loc[sf_merged['Cluster Label'] == 10, sf_merged.columns[[0] + list(range(3, sf_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
